In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The firefly algorithm

In [2]:
def firefly_algorithm(
        func, rnge,
        pop_size=30,
        generations=200,
        beta0=1.0,
        gamma=1.0,
        alpha=0.25,
        alpha_decay=0.97):
    # Initialize population and evaluate initial brightness (fitness)
    lower, upper = rnge
    positions = np.random.uniform(lower, upper, size=(pop_size, 2))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    best_index = np.argmin(fitness)
    best_position = positions[best_index].copy()
    best_value = fitness[best_index]
    improvement_id = 0
    yield "g0000_i00000", tuple(best_position)
    # Main generational loop
    for generation in range(1, generations + 1):
        # Pairwise interactions: move dimmer fireflies toward brighter ones
        for i in range(pop_size):
            for j in range(pop_size):
                if fitness[j] < fitness[i]:
                    difference = positions[j] - positions[i]
                    distance_sq = np.dot(difference, difference)
                    beta = beta0 * np.exp(-gamma * distance_sq)
                    random_step = np.random.uniform(-0.5, 0.5, size=2) * (upper - lower)
                    candidate = positions[i] + beta * difference + alpha * random_step
                    candidate = np.clip(candidate, lower, upper)
                    candidate_value = func(tuple(candidate))
                    # Always apply the update (no greedy acceptance gate)
                    positions[i] = candidate
                    fitness[i] = candidate_value
                    if candidate_value < best_value:
                        best_value = candidate_value
                        best_position = candidate.copy()
                        improvement_id += 1
                        yield f"g{generation:04d}_i{improvement_id:05d}", tuple(best_position)
        # Randomization parameter decay and global-best check
        alpha *= alpha_decay
        current_best_index = np.argmin(fitness)
        if fitness[current_best_index] < best_value:
            best_value = fitness[current_best_index]
            best_position = positions[current_best_index].copy()
            improvement_id += 1
            yield f"g{generation:04d}_i{improvement_id:05d}", tuple(best_position)

#### Run the firefly algorithm and collect frames for multiple functions

In [3]:
gif_folder_path = "gifs/firefly"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for idx, point in firefly_algorithm(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=idx)
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}_{idx}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the firefly algorithm on multiple functions

![Sphere](gifs/firefly/Sphere.gif)
![Ackley](gifs/firefly/Ackley.gif)
![Rastrigin](gifs/firefly/Rastrigin.gif)
![Rosenbrock](gifs/firefly/Rosenbrock.gif)
![Griewank](gifs/firefly/Griewank.gif)
![Schwefel](gifs/firefly/Schwefel.gif)
![Levy](gifs/firefly/Levy.gif)
![Michalewicz](gifs/firefly/Michalewicz.gif)
![Zakharov](gifs/firefly/Zakharov.gif)